In [1]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
CV = CountVectorizer()
from scipy.sparse import csr_matrix
from PodcastFeatureEngineering import Podcast_Features
import graphlab as gl
from graphlab import SFrame as sf
import time

In [2]:
client = MongoClient()
# Access/Initiate Database
db = client['Podcast']
# Access/Initiate Table
Podcast_twitter = db['PodcastTwitter_handles']
print Podcast_twitter.count()

153497


In [3]:
practice = []
n=0
for pod in Podcast_twitter.find({}, {'Title':1, '_id':0, 'TwitterFollowers':1}):
    try:
        if 'TwitterFollowers' in pod and pod['TwitterFollowers'] and pod['TwitterFollowers'][0]!='None':
            practice.append(pod)
            n+=1
    except:
        pass
print n

18918


In [4]:
import json
aws = json.load(open("/home/ubuntu/.api/AWS.api"))
key = str(aws['AWS_ACCESS_KEY'])
secret = str(aws['AWS_SECRET_ACCESS_KEY'])
gl.aws.set_credentials(access_key_id = key, secret_access_key = secret)
my_config = gl.deploy.Ec2Config()

In [5]:
my_cluster = gl.deploy.ec2_cluster.create('Cluster_3',
                                                     's3://galvanizecrystal',
                                                     my_config,
                                                     num_hosts = 4)

2016-04-17 00:03:33,638 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.8.5 started. Logging: /tmp/graphlab_server_1460851412.log
2016-04-17 00:03:43,813 [INFO] graphlab.connect.aws._ec2, 503: Launching an m3.xlarge instance in the us-west-2b availability zone, with id: i-55362192. You will be responsible for the cost of this instance.
2016-04-17 00:04:01,380 [INFO] graphlab.deploy._executionenvironment, 364: 3 additional hosts will be launched by i-55362192
2016-04-17 00:04:01,382 [INFO] graphlab.deploy._executionenvironment, 367: Waiting for i-55362192 to start up.


This non-commercial license of GraphLab Create is assigned to crystal.humphries@systemsbiology.org and will expire on November 04, 2016. For commercial licensing options, visit https://dato.com/buy/.


RuntimeError: Unable to start host(s). Please terminate manually from the AWS console.

In [4]:
def create_sparse_matrix(list_of_followers):
    list_of_strings = []
    list_of_titles = []
    for ls in list_of_followers:
        x = ' '.join([ str(x) for x in ls['TwitterFollowers']])
        list_of_strings.append(x)
        list_of_titles.append(ls['Title'])
    return list_of_strings, list_of_titles

In [5]:
strings,titles = create_sparse_matrix(practice)
final = CV.fit_transform(strings)
a = csr_matrix.sum(final, axis=0)
print final.shape
a = np.array(a).reshape(a.shape[1],)
mask = a>1

u=np.where(mask)[0]
sparse_mat = final.tocsc()[:,u]
mat = sparse_mat.toarray()

(17906, 2046770)


In [6]:
def get_truncated_matrix(mat, max_num):
    df = pd.DataFrame(mat[:,0:max_num])
    df.index = titles
    a = df.sum(axis=1)
    mask2 = a>0
    return df.loc[mask2,:]

In [31]:
start = time.time()
attempt = get_truncated_matrix(mat, 2028372)
print attempt.shape
end = time.time()
print(end - start)

(17250, 243629)
15.593239069


In [7]:
df_total = pd.DataFrame(mat)
df_total.index = titles
df_total.shape

(17906, 243629)

In [8]:
PF = Podcast_Features()

/home/ubuntu/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:3066: DtypeWarning: Columns (14,15,17) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
clean_podcast_csv.py:24: FutureWarning: the coerce=True keyword is deprecated, use errors='coerce' instead
  self.df['DateFirstEpisode'] = self.df.DateFirstEpisode.apply(lambda x: pd.to_datetime(str(x), coerce=True))


Creating Tf-idf


clean_podcast_csv.py:25: FutureWarning: the coerce=True keyword is deprecated, use errors='coerce' instead
  self.df['DateLastEpisode']  = self.df.DateLastEpisode.apply(lambda x: pd.to_datetime(str(x), coerce=True))


In [9]:
df = PF.cleaned_df

In [29]:
# gl.deploy.ec2_cluster.create('Compute Cluster', 
#                              's3://crystalgalvanize/',
#                                 my_config,
#                                                      num_hosts = 4)

In [10]:
g = df.columns.to_series().groupby(df.dtypes).groups
c = g.keys()

In [11]:
df2 = df.drop(g[c[0]], axis=1)
df2 = df2.drop(g[c[1]], axis=1)

In [13]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 155804 entries, 0 to 155803
Data columns (total 5 columns):
TotalEpisodes      155804 non-null int64
RatingCount        155804 non-null int64
AverageRating      155804 non-null float64
CategoryRanking    155804 non-null int64
Title              155804 non-null object
dtypes: float64(1), int64(3), object(1)
memory usage: 7.1+ MB


In [12]:
df2['Title'] = df.Title

In [41]:
keep_from_df2 = df2.Title.isin(attempt.index) #& df_total.index.isin(df2.Title)
keep_from_df_total = attempt.index.isin(df2.Title)

In [44]:
items = df2.loc[keep_from_df2,:]
#users = attempt.loc[keep_from_df_total,:]

In [19]:
sf_items = gl.SFrame(items) 

In [45]:
users.shape, items.shape

((16038, 243629), (16343, 5))

In [ ]:
users2 = gl.SFrame(users)

In [38]:
gl.recommender.item_similarity_recommender.create

In [48]:
users.save("users_podcast_sframe.gl")

AttributeError: 'DataFrame' object has no attribute 'save'

In [50]:
gl.SFrame()

pandas.core.frame.DataFrame

In [55]:
df_total.shape

(17723, 240254)

In [87]:
df_total.shape

(17794, 241246)